In [1]:
import pandas as pd
import torch

In [2]:
with open('mh.txt') as f:
    data = f.read()
    print(data)




CHAPTER 1




Sources for the History of Modern India
An abundance of historical material is available for studying India from the mid-18th century to the mid-20th century. In constructing the history of modern India, priority needs to be given to the archives. Archives refer to a collection of historical records and documents, usually primary source documents, i.e., those documents that have been created as a necessary part of some activity—administrative, legal, social, or commercial. They are unique/original documents, not consciously written or created to convey information to a future generation. An important part of archives relating to modern India are the official records, i.e., the papers of government agencies at various levels.
1
The records of the East India Company provide a detailed account of trading conditions during the period 1600–1857. When the British crown took over the administration, it also kept a large variety and volume of official records. These records he

In [2]:
# You can use this section to suppress warnings generated by your code:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn
warnings.filterwarnings('ignore')

from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory

In [10]:
from collections import defaultdict
import spacy
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.schema import Document
from langchain.vectorstores import Chroma
import random
import en_core_web_trf
from pprint import pprint
from lmqg import TransformersQG
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.schema import Document

class MCQGeneration:
    def __init__(self, file):
        # Initialize model and embeddings
        self.model = TransformersQG(model="lmqg/t5-base-squad-qag")
        self.embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
        self.persons = []

        # Load SpaCy model (ensure it's installed)
        try:
            self.nlp = spacy.load("en_core_web_trf")
        except OSError:
            print("Downloading SpaCy model 'en_core_web_trf'...")
            from spacy.cli import download
            download("en_core_web_trf")
            self.nlp = spacy.load("en_core_web_trf")

        # Read input file
        with open(file, "r") as f:
            self.data = f.read()

    def ner_extraction(self):
        """Extract named entities and generate documents."""
        doc = self.nlp(self.data)
        words = defaultdict(set)

        # Extract entities from sentences
        for sent in doc.sents:
            for ent in sent.ents:
                words[ent.text].add((ent.text, sent.text, ent.label_))

        context_answers = [item for value in words.values() for item in value]
        persons = [(i[1], i[0], i[2]) for i in context_answers]

        # Create documents with metadata
        documents = [
            Document(page_content=context, metadata={"ner_tag": ner_tag, "class": label})
            for context, ner_tag, label in persons
        ]
        return documents, persons

    def search(self, documents, persons):
        """Perform similarity search and generate MCQs."""
        docsearch = Chroma.from_documents(documents, self.embeddings)
        print("Documents ingested successfully!")

        for doc in documents:
            query = doc.page_content
            results = docsearch.similarity_search(query)
            linked = {}
            question_answer = self.model.generate_qa(query)

            # Use a temporary dictionary to avoid modifying during iteration
            temp_linked = {}
            for i in question_answer:
                linked[i] = i
                temp_linked[i] = i

            for keys, values in temp_linked.items():
                query = values[0]
                ner = values[1]

                # Filter persons based on NER tag
                x = [j[2] for j in persons if j[1] == ner]

                results = docsearch.similarity_search(query, k=100)  # Retrieve top-100 results
                options = []

                for result in results:
                    print(result.metadata.get("class"))
                    if result.metadata.get("class") in x:
                        options.append(result.metadata.get("ner_tag"))

                # Handle cases where fewer than 3 options are available
                if len(options) < 3:
                    options += [None] * (3 - len(options))  # Pad with None if not enough options

                if len(options) >= 3:  # Ensure there are enough options to sample from
                    random_values = random.sample(options, 3)
                    random_values.append(question_answer[0][1])  # Add the correct answer
                    random.shuffle(random_values)  # Shuffle the options
                    linked["options"] = random_values

        yield linked





In [3]:
mcq = MCQGeneration('mh.txt')

NameError: name 'MCQGeneration' is not defined

In [12]:
documents,persons = mcq.ner_extraction()

In [15]:
all_results = []
h = 0

# Consume the generator and collect results
for linked in mcq.search(documents, persons):  # Ensure `mcq` is defined and initialized
    if h >= 1:  # Stop collecting after 10 outputs
        break
    all_results.append(linked)  # Append each yielded result to the list
    h += 1

# Print the results to verify
print(all_results)


Documents ingested successfully!


100%|██████████| 1/1 [00:00<00:00, 1372.93it/s]


[('When were the demands for constitutional reforms centred around?', '1885 to 1892'), ('What did the nationalist demands for constitutional reforms center around?', '1. expansion of councils'), ('What did the nationalist demands for constitutional reforms focus on?', 'greater participation ofIndians in councils'), ('What did the nationalist demands for constitutional reforms focus on?', 'more powers to councils'), ('What did the nationalist demands for constitutional reforms focus on?', 'greater control over finances')]
CARDINAL
CARDINAL
DATE
NORP
DATE
NORP
CARDINAL
CARDINAL
NORP
CARDINAL
DATE
CARDINAL
DATE
NORP
CARDINAL
CARDINAL
ORG
NORP
ORG
NORP
ORG
NORP
ORG
NORP
ORG
ORG
ORG
ORG
LAW
DATE
DATE
LAW
LAW
DATE
LAW
DATE
DATE
ORG
ORG
DATE
DATE
ORG
DATE
ORG
ORG
PERSON
PERSON
PERSON
DATE
PERSON
PERSON
ORG
PERSON
PERSON
PERSON
DATE
PERSON
PERSON
PERSON
PERSON
PERSON
PERSON
PERSON
ORG
DATE
DATE
PERSON
PERSON
PERSON
PERSON
PERSON
ORG
NORP
GPE
GPE
NORP
NORP
GPE
NORP
GPE
LAW
PERSON
DATE
DATE
LAW


100%|██████████| 1/1 [00:00<00:00, 1517.48it/s]


[('What did the Indian Civil Service protest against in 1877?', 'the reduction of age limit'), ('What did the Indian Civil Service protest against in 1877?', 'the reduction of age limit for candidates of the Indian Civil Service examination'), ('What did the Indian Civil Service protest against in 1877?', 'the reduction of age limit for candidates of the Indian Civil Service examination')]
DATE
CARDINAL
ORG
NORP
CARDINAL
ORG
NORP
CARDINAL
CARDINAL
DATE
NORP
CARDINAL
DATE
ORG
CARDINAL
CARDINAL
DATE
NORP
ORG
CARDINAL
GPE
ORG
LAW
ORG
LAW
ORG
CARDINAL
DATE
LAW
CARDINAL
DATE
ORG
NORP
CARDINAL
CARDINAL
CARDINAL
NORP
DATE
ORG
DATE
CARDINAL
DATE
ORG
LAW
CARDINAL
ORG
NORP
DATE
DATE
ORG
CARDINAL
LAW
DATE
CARDINAL
ORG
GPE
ORG
CARDINAL
CARDINAL
ORG
DATE
ORG
DATE
LAW
CARDINAL
LAW
LAW
CARDINAL
NORP
CARDINAL
CARDINAL
ORG
NORP
CARDINAL
LAW
GPE
ORG
DATE
CARDINAL
ORG
LAW
CARDINAL
ORG
LAW
ORG
NORP
LAW
DATE
DATE
DATE
CARDINAL
CARDINAL
ORG
CARDINAL
DATE
DATE
ORG
DATE
ORG
NORP
DATE
CARDINAL
ORG
NORP
CARDINA

...

100%|██████████| 1/1 [00:00<00:00, 2799.94it/s]


[('Who was the originator of the practice of subsidiary alliance in India?', 'Dupleix'), ('Who was the originator of the practice of subsidiary alliance in India?', 'Dupleix'), ('Who was the originator of the practice of subsidiary alliance in India?', 'Dupleix')]
NORP
ORG
Matched Context: Besides, the subsidiary alliance would expand
the Company’s hold over the Indian states and gradually bring more and more territory into the Company’s fold.
NER Tag: Company
label: ORG
ORG
Matched Context: Besides, the subsidiary alliance would expand
the Company’s hold over the Indian states and gradually bring more and more territory into the Company’s fold.
NER Tag: Company
label: ORG
NORP
ORG
Matched Context: Besides, the subsidiary alliance would expand
the Company’s hold over the Indian states and gradually bring more and more territory into the Company’s fold.
NER Tag: Company
label: ORG
NORP
NORP
ORG
Matched Context: Besides, the subsidiary alliance would expand
the Company’s hold over the In

100%|██████████| 1/1 [00:00<00:00, 1508.20it/s]


[('Who assassinated Curzon-Wyllie?', 'Madanlal Dhingra'), ('When was Curzon-Wyllie assassinated?', '1909'), ('When was Savarkar extradited?', '1910'), ('Who was transported for life in the Nasik conspiracy case?', 'Savarkar')]
DATE
DATE
DATE
DATE
PERSON
Matched Context: Wazir Ali’s guards killed two other Europeans and even attacked the Magistrate of Benares.
NER Tag: Wazir Ali’s
label: PERSON
CARDINAL
GPE
NORP
CARDINAL
NORP
PERSON
Matched Context: Wazir Ali’s guards killed two other Europeans and even attacked the Magistrate of Benares.
NER Tag: Wazir Ali’s
label: PERSON
GPE
CARDINAL
GPE
PERSON
Matched Context: Wazir Ali’s guards killed two other Europeans and even attacked the Magistrate of Benares.
NER Tag: Wazir Ali’s
label: PERSON
NORP
CARDINAL
PERSON
Matched Context: Wazir Ali’s guards killed two other Europeans and even attacked the Magistrate of Benares.
NER Tag: Wazir Ali’s
label: PERSON
NORP
GPE
GPE
PERSON
Matched Context: In April 1816, the insurgents murdered the son of Ley

KeyboardInterrupt: 

In [4]:
import spacy
import en_core_web_trf
from collections import defaultdict

nlp = en_core_web_trf.load()

# Load the pre-trained model
nlp = spacy.load('en_core_web_trf')
doc = nlp(data)
words = defaultdict(set)
for sent in doc.sents:
    for ent in sent.ents:
        words[ent.label_].add((ent.text,sent.text,ent.label_)) 
        print(ent.text, ent.label_)

/home/kronos/anaconda3/envs/pytorch_env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


1 CARDINAL
India GPE
the mid-18th century to the mid-20th century DATE
India GPE
India GPE
the East India Company ORG
the period 1600–1857. DATE
British NORP
European NORP
East India GPE
Portuguese NORP
Dutch NORP
French NORP
the 17th and 18th centuries DATE
the 18th and early 19th centuries DATE
the 18th century DATE
Indian NORP
the 19th and 20th centuries DATE
Indian NORP
four CARDINAL
ii CARDINAL
Central Government Archives ORG
The National Archives of India ORG
New Delhi GPE
the Government of India ORG
Indian NORP
the National Archives ORG
the  East  India  Company ORG
James Rennell PERSON
first ORDINAL
1767 DATE
the Survey of India ORG
the Survey of India ORG
1907 DATE
the Reforms Office ORG
1920 to 1937 DATE
the State Governments ORG
British Indian NORP
ii CARDINAL
the Indian Union ORG
1947 DATE
iii CARDINAL
British NORP
Indian NORP
British NORP
the Kingdom of Lahore GPE
Khalsa Darbar GPE
1800 to 1849 DATE
pre-British NORP
India GPE
the Alienation Office ORG
Pune GPE
Maratha NORP

In [5]:
words.keys()

dict_keys(['CARDINAL', 'GPE', 'DATE', 'ORG', 'NORP', 'PERSON', 'ORDINAL', 'FAC', 'LOC', 'LANGUAGE', 'WORK_OF_ART', 'EVENT', 'LAW', 'QUANTITY', 'PRODUCT', 'MONEY', 'PERCENT', 'TIME'])

In [6]:
context_answers = []
for key, value in words.items():
    for i in words[key]:
        context_answers.append(i)
persons = [(i[1],i[0],i[2]) for i in context_answers]

In [7]:
context_answers

[('Abdali',
  'Before his return, Abdali had recognised Alamgir II as the Mughal emperor and the Rohilla chief, Najib- ud-Daula, as Mir Bakhshi of the empire, who was to act as personal ‘supreme agent’ of Abdali.',
  'PERSON'),
 ('Asaf-ud-Daula',
  'Asaf-ud-Daula, in 1784, built Bara Imambara at Lucknow.',
  'PERSON'),
 ('Sher Singh',
  'Sher Singh was sent to suppress the revolt, but he himself joined Mulraj, leading to a mass uprising  in Multan.',
  'PERSON'),
 ('Vasudev Balwant Phadke',
  'The first of the revolutionary activities in Maharashtra was the organisation of the Ramosi Peasant Force by Vasudev Balwant Phadke in 1879, which aimed to rid the country of the British by instigating an armed revolt by disrupting communication lines.',
  'PERSON'),
 ('Devi',
  'The Devi movement was initially a social movement among the tribals of South Gujarat in 1922-23.',
  'PERSON'),
 ('Shaukat Usmani',
  '\nIn 1924, many communists—S.A. Dange, Muzaffar Ahmed, Shaukat Usmani, Nalini Gupta—w

In [8]:
[i[2] for i in persons if i[1] == 'Santhal Pargana' for x in i]

[]

In [11]:
from langchain.schema import Document

# Example: persons is a list of tuples [(context, ner_tag), ...]


# Convert to Document objects
documents = [
    Document(page_content=context, metadata={"ner_tag": ner_tag,"class":label})
    for context, ner_tag, label in persons
]


In [12]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.schema import Document
# Initialize embeddings model
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

docsearch = Chroma.from_documents(documents, embeddings)

print("Documents ingested successfully!")


Documents ingested successfully!


In [13]:
persons[1][0]

'Asaf-ud-Daula, in 1784, built Bara Imambara at Lucknow.'

In [14]:
# Perform a similarity search
query = 'The burning of foreign cloth by Gandhi and the burning of Manusmriti by Ambedkar are not to be seen as mere acts of sentiment. Rather, foreign cloth and Manusmriti represented the bondage and slavery for India. So too, a pinch of salt from the ocean, and a drop of water from the Mahad tank were acts of political catharsis and social philosophy.'
results = docsearch.similarity_search(query, k=50)  # Retrieve top-1 result

for result in results:
    print("Matched Context:", result.page_content)
    print("NER Tag:", result.metadata.get("ner_tag"))
    print("NER label:", result.metadata.get("class"))

Matched Context: The burning of foreign cloth by Gandhi and the burning of Manusmriti by Ambedkar are not to be seen as mere acts of sentiment.
NER Tag: Ambedkar
NER label: PERSON
Matched Context: The burning of foreign cloth by Gandhi and the burning of Manusmriti by Ambedkar are not to be seen as mere acts of sentiment.
NER Tag: Gandhi
NER label: PERSON
Matched Context: The burning of foreign cloth by Gandhi and the burning of Manusmriti by Ambedkar are not to be seen as mere acts of sentiment.
NER Tag: Manusmriti
NER label: PERSON
Matched Context: Later, in December 1927, Ambedkar and his colleagues burnt the ‘Manusmriti’ at the same place as a gesture of getting rid of inequalities.
NER Tag: Ambedkar
NER label: PERSON
Matched Context: Gandhi wanted people to get rid of greed and make do with just the bare necessities of life.
NER Tag: Gandhi
NER label: PERSON
Matched Context: The Indians treated this judgement as an insult to the honour of Tolstoy Farm  
As it became rather difficu

doc = text

In [15]:
example = ('the Santhal Revolt',
               'It was after the Santhal Revolt of 1855-56 that the Santhal Pargana was created out of the districts of Bhagalpur and Birbhum.')

In [128]:
context = '''Sher Singh was sent to suppress the revolt, but he himself joined Mulraj, leading to a mass uprising  in Multan.'''
entity = example[0]

In [17]:
from pprint import pprint
from lmqg import TransformersQG
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.schema import Document

# initialize model
model = TransformersQG(model='lmqg/t5-base-squad-qag') # or TransformersQG(model='lmqg/t5-base-squad-qg-ae') 
# paragraph to generate pairs of question and answer


/home/kronos/anaconda3/envs/pytorch_env/lib/python3.12/site-packages/transformers/models/auto/tokenization_auto.py:809: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
/home/kronos/anaconda3/envs/pytorch_env/lib/python3.12/site-packages/transformers/models/auto/configuration_auto.py:1002: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
/home/kronos/anaconda3/envs/pytorch_env/lib/python3.12/site-packages/transformers/modeling_utils.py:3479: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


/home/kronos/anaconda3/envs/pytorch_env/lib/python3.12/site-packages


In [129]:
question_answer = model.generate_qa(context)
# the output is a list of tuple (question, answer)


100%|██████████| 1/1 [00:00<00:00, 595.36it/s]


In [130]:
question_answer

[('Who was sent to suppress the revolt?', 'Sher Singh'),
 ('Who joined Mulraj?', 'Sher Singh'),
 ('What led to a mass uprising in Multan?', 'Sher Singh')]

In [131]:
linked = []


In [132]:
solution = {}
for i in question_answer:
    linked.append(i)
    



In [133]:
linked


[('Who was sent to suppress the revolt?', 'Sher Singh'),
 ('Who joined Mulraj?', 'Sher Singh'),
 ('What led to a mass uprising in Multan?', 'Sher Singh')]

In [134]:
for i in linked:
    print(i)

('Who was sent to suppress the revolt?', 'Sher Singh')
('Who joined Mulraj?', 'Sher Singh')
('What led to a mass uprising in Multan?', 'Sher Singh')


In [135]:
persons

[('Before his return, Abdali had recognised Alamgir II as the Mughal emperor and the Rohilla chief, Najib- ud-Daula, as Mir Bakhshi of the empire, who was to act as personal ‘supreme agent’ of Abdali.',
  'Abdali',
  'PERSON'),
 ('Asaf-ud-Daula, in 1784, built Bara Imambara at Lucknow.',
  'Asaf-ud-Daula',
  'PERSON'),
 ('Sher Singh was sent to suppress the revolt, but he himself joined Mulraj, leading to a mass uprising  in Multan.',
  'Sher Singh',
  'PERSON'),
 ('The first of the revolutionary activities in Maharashtra was the organisation of the Ramosi Peasant Force by Vasudev Balwant Phadke in 1879, which aimed to rid the country of the British by instigating an armed revolt by disrupting communication lines.',
  'Vasudev Balwant Phadke',
  'PERSON'),
 ('The Devi movement was initially a social movement among the tribals of South Gujarat in 1922-23.',
  'Devi',
  'PERSON'),
 ('\nIn 1924, many communists—S.A. Dange, Muzaffar Ahmed, Shaukat Usmani, Nalini Gupta—were jailed in the Ka

In [136]:
all = []
for i in linked:
    query=(i[0])
    print(query)
    ner = (i[1])
    results = docsearch.similarity_search(query, k=5)  # Retrieve top-1 result
    x = [j[2] for j in persons if j[1] == ner]
    print(x)
    options = []
    for result in results:
        if result.metadata.get("class") in x and result.metadata.get("ner_tag") not in options:
            options.append(result.metadata.get("ner_tag"))
    all.append((query,ner,options))

Who was sent to suppress the revolt?
['PERSON', 'PERSON', 'PERSON']
Who joined Mulraj?
['PERSON', 'PERSON', 'PERSON']
What led to a mass uprising in Multan?
['PERSON', 'PERSON', 'PERSON']


In [137]:
all

[('Who was sent to suppress the revolt?',
  'Sher Singh',
  ['S.B. Chaudhuri', 'Mulraj', 'Sher Singh', 'Neill', 'Tara Chand']),
 ('Who joined Mulraj?',
  'Sher Singh',
  ['Mulraj', 'Sher Singh', 'R.G. Bhandarkar', 'N.G. Chandavarkar']),
 ('What led to a mass uprising in Multan?',
  'Sher Singh',
  ['Mulraj', 'Sher Singh', 'Birsa Munda', 'Munda Uprisings'])]

In [138]:
valid = []
for i in all:
    if i[2] != []:
        valid.append(i)
valid

[('Who was sent to suppress the revolt?',
  'Sher Singh',
  ['S.B. Chaudhuri', 'Mulraj', 'Sher Singh', 'Neill', 'Tara Chand']),
 ('Who joined Mulraj?',
  'Sher Singh',
  ['Mulraj', 'Sher Singh', 'R.G. Bhandarkar', 'N.G. Chandavarkar']),
 ('What led to a mass uprising in Multan?',
  'Sher Singh',
  ['Mulraj', 'Sher Singh', 'Birsa Munda', 'Munda Uprisings'])]

In [139]:
import json
import random
output = []
for question, correct_answer, options in valid:
    # Remove the correct answer from options
    filtered_options = [opt for opt in options if opt != correct_answer]
    
    # Randomly select 3 incorrect answers
    random_choices = random.sample(filtered_options, 3)
    
    # Add the correct answer and shuffle
    random_choices.append(correct_answer)
    random.shuffle(random_choices)
    
    # Append formatted question set to output
    output.append({
        "question": question,
        "correctAnswer": correct_answer,
        "options": random_choices
    })

# Write to JSON file
with open("questions.json", "w") as f:
    json.dump(output, f, indent=4)

print(output)


[{'question': 'Who was sent to suppress the revolt?', 'correctAnswer': 'Sher Singh', 'options': ['S.B. Chaudhuri', 'Sher Singh', 'Tara Chand', 'Mulraj']}, {'question': 'Who joined Mulraj?', 'correctAnswer': 'Sher Singh', 'options': ['Mulraj', 'N.G. Chandavarkar', 'Sher Singh', 'R.G. Bhandarkar']}, {'question': 'What led to a mass uprising in Multan?', 'correctAnswer': 'Sher Singh', 'options': ['Munda Uprisings', 'Sher Singh', 'Mulraj', 'Birsa Munda']}]


In [84]:
solution

{'question': 'What did a pinch of salt from the ocean and a drop of water from the Mahad tank represent?',
 'correctAnswer': 'political catharsis and social philosophy',
 'options': ['Gandhi',
  'C. Rajagopalachari',
  'Emperor',
  'Gopalbandhu Choudhuri']}

In [137]:
linked
    

<generator object MCQGeneration.search at 0x7fa5dd7f0770>

In [145]:
r = {}

In [148]:
r[1] = 'x'

In [162]:
r[2] = 'y'

In [165]:
for keys,values in r.items():
    print(values)

x
y


In [168]:
for key,values in r.items():
    for i in range(len(values)):
        h = values
        print(h)

x
y


In [156]:
print(r.keys)

<built-in method keys of dict object at 0x7fa5f1a3ca80>
